<a href="https://colab.research.google.com/github/amylynnn/weatherwise-Amylynn-Sophie/blob/main/AmylynnSophieWeatherWise__starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise



In [ ]:
API_KEY = '7cf20335110caaf78db0fecb31852d45' #7cf20335110caaf78db0fecb31852d45
# Simple in-memory cache: { (source, city): (timestamp, data) }
weather_cache = {}
# A dictionary that stores cached weather data to avoid repeated API calls.
CACHE_EXPIRY_MINUTES = 10  # Cache duration

## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests                # For HTTP requests to weather APIs
import matplotlib.pyplot as plt # For plotting weather charts
import ipywidgets as widgets    # For interactive UI elements in Jupyter/Colab
from IPython.display import display, clear_output # To display widgets and clear output areas
import re                      # For parsing user questions with regular expressions
from datetime import datetime, timedelta # For handling dates and times
from random import choice      # For picking random humorous messages

## 🌤️ Weather Data Functions

In [ ]:
# Define get_weather_data() function here
def fetch_weather_data(city, source='openweathermap'):
    """Fetch weather data for a city, using cache if available and valid."""
    # Key is (source, city) to support multiple sources and cities in the cache
    key = (source.lower(), city.lower())
    now = datetime.utcnow()
    # Use cached data if it's still fresh (not expired)
    if key in weather_cache:
        timestamp, cached_data = weather_cache[key]
        if now - timestamp < timedelta(minutes=CACHE_EXPIRY_MINUTES):
            # Return cached data to avoid unnecessary API calls
            return cached_data
    # Otherwise, fetch new data from the specified source
    if source.lower() == 'openweathermap':
        data = fetch_openweathermap(city)  # Fetch from OpenWeatherMap API
    elif source.lower() == 'mocksource':
        data = fetch_mock_source(city)      # Fetch from mock (random) data source
    else:
        # Raise an error if the source is not supported
        raise ValueError(f"Unsupported weather data source: {source}")
    # Store the new data in the cache with the current timestamp
    weather_cache[key] = (now, data)
    return data

def fetch_openweathermap(city):  # Calls the actual weather API and returns the JSON forecast.
    """Call OpenWeatherMap API to get weather forecast for the given city."""
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)           # Make HTTP GET request to the API
    response.raise_for_status()            # Raise an error if the request failed
    return response.json()                 # Return the parsed JSON data

def fetch_mock_source(city):
    """Generate random mock weather data for testing/demo."""
    from random import uniform, randint
    now = datetime.utcnow()
    list_data = []
    for i in range(40):  # 5 days, 3-hour intervals (8 intervals per day)
        dt = now + timedelta(hours=3*i)   # Calculate the datetime for each interval
        dt_txt = dt.strftime("%Y-%m-%d %H:%M:%S")  # Format as string
        temp = round(uniform(10, 25), 1)           # Generate random temperature
        rain = round(uniform(0, 5), 1) if randint(0, 1) else 0  # Random rain value or 0
        snow = round(uniform(0, 2), 1) if randint(0, 3) == 0 else 0  # Random snow value or 0
        weather_desc = 'clear sky' if randint(0, 2) == 0 else 'rain' # Random weather description
        entry = {
            'dt_txt': dt_txt,                             # Date and time as string
            'main': {'temp': temp},                       # Temperature
            'rain': {'3h': rain} if rain > 0 else {},     # Rain volume (if any)
            'snow': {'3h': snow} if snow > 0 else {},     # Snow volume (if any)
            'weather': [{'description': weather_desc}],   # Weather description
            'wind': {'speed': round(uniform(0, 10),1), 'deg': randint(0,360)} # Wind info
        }
        list_data.append(entry)
    return {'list': list_data}  # Creates fake weather data for testing the app without hitting the API.


## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here

def interactive_temperature_plot(data, city):
    # I used 'interactive' to let user explore temperature forecast dynamically with a slider
    """Show an interactive temperature line chart with a slider."""

    # Extract datetime objects from the forecast data timestamps
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]

    # Extract temperature values corresponding to each timestamp
    temps = [entry['main']['temp'] for entry in data['list']]

    # Inner function to plot temperature up to a certain index (controlled by slider)
    def plot_upto(index=8):
        plt.figure(figsize=(12, 6))  # Set figure size
        plt.plot(times[:index], temps[:index], marker='o', linestyle='-', color='orange')  # Plot line graph
        plt.title(f"Temperature Forecast for {city.title()}")  # Chart title with city name
        plt.xlabel("Date & Time")  # X-axis label
        plt.ylabel("Temperature (°C)")  # Y-axis label
        plt.xticks(rotation=45)  # Rotate x-axis labels for readability
        plt.grid(True)  # Enable grid for better visualization
        plt.tight_layout()  # Adjust layout to prevent clipping
        plt.show()  # Display the plot

    # Create an interactive slider widget to control how many data points to show
    widgets.interact(plot_upto, index=widgets.IntSlider(min=1, max=len(times), step=1, value=8))





In [ ]:
def interactive_precipitation_plot(data, city):
    # I used 'interactive' to let users explore the rain forecast dynamically with a slider.
    """Show an interactive precipitation bar chart with a slider."""

    # Extract datetime objects from forecast timestamps
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]

    # Extract rain volume (mm) for each 3-hour period; default to 0 if no rain data
    rain = [entry.get('rain', {}).get('3h', 0) for entry in data['list']]

    # Inner function to plot precipitation bars up to a given index controlled by slider
    def plot_upto(index=8):
        plt.figure(figsize=(12, 6))  # Set figure size
        plt.bar(times[:index], rain[:index], width=0.1, color='blue', edgecolor='black')  # Bar chart
        plt.title(f"Precipitation Forecast for {city.title()} (Next 5 Days)")  # Title with city name
        plt.xlabel("Date & Time")  # X-axis label
        plt.ylabel("Rain (mm in 3h)")  # Y-axis label
        plt.xticks(rotation=45)  # Rotate x-axis labels for readability
        plt.grid(axis='y', linestyle='--', alpha=0.5)  # Horizontal grid lines with light style
        plt.tight_layout()  # Adjust layout to prevent clipping
        plt.show()  # Display the plot

    # Create interactive slider widget to control how many bars to show
    widgets.interact(plot_upto, index=widgets.IntSlider(min=1, max=len(times), step=1, value=8))


## 🤖 Natural Language Processing

In [ ]:
# Extracts the user's intent (rain, temperature, etc.), city, and time frame from their question.
def parse_weather_question(question):
    """Parse user's question to extract weather condition, city, and time frame."""
    question_lower = question.lower()

    # Dictionary mapping weather conditions to their possible synonyms in user queries
    condition_keywords = {
        'rain': [...], 'snow': [...], 'clear': [...], 'cloudy': [...],
        'temperature': [...], 'wind': [...], 'humidity': [...], 'storm': [...], 'fog': [...]
    }

    # Detect which weather condition is mentioned in the question
    condition = None
    for cond, synonyms in condition_keywords.items():
        for syn in synonyms:
            if syn in question_lower:
                condition = cond
                break
        if condition:
            break
    # Default to 'general' if no specific condition found
    if not condition:
        condition = 'general'

    # Detect time frame mentioned in the question (e.g., today, tomorrow, next N days)
    if 'day after tomorrow' in question_lower:
        time_frame = 'day_after_tomorrow'
    elif 'tomorrow' in question_lower:
        time_frame = 'tomorrow'
    elif 'today' in question_lower:
        time_frame = 'today'
    else:
        # Check for "next N days" pattern
        match = re.search(r'next (\d+) days', question_lower)
        if match:
            time_frame = f"next_{match.group(1)}_days"
        elif 'this weekend' in question_lower:
            time_frame = 'this_weekend'
        else:
            # Default to today if no time frame specified
            time_frame = 'today'

    # Try to extract city name following the word "in"
    city_match = re.search(
        r"in ([a-zA-ZÀ-ÿ\s\-]+?)(?:\s+(?:today|tomorrow|day after tomorrow|next \d+ days|this weekend)|\?|$| ... )",
        question_lower)
    if city_match:
        city = city_match.group(1).strip()
    else:
        # Fallback: extract last capitalized word(s) as city name
        capitalized = re.findall(r"\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*", question)
        city = capitalized[-1] if capitalized else None

    # Normalize some common city names to include country code for better API accuracy
    fixes = {...}
    if city:
        city_key = city.lower()
        city = fixes.get(city_key, city.title())
    else:
        city = None

    # Return parsed components as a dictionary
    return {
        'condition': condition,
        'city': city,
        'time_frame': time_frame
    }


## 🧭 User Interface

In [ ]:
def interactive_ui():

    # Input widgets for user to enter city, ask a question, and select data source
    city_input = widgets.Text(...)         # Text box for city name input
    question_input = widgets.Text(...)     # Text box for weather-related question
    source_dropdown = widgets.Dropdown(...) # Dropdown to select weather data source

    # Output widget area where responses and charts will be displayed
    output_area = widgets.Output(...)

    # Buttons for showing forecast charts, answering questions, and clearing inputs/output
    forecast_btn = widgets.Button(...)
    question_btn = widgets.Button(...)
    clear_btn = widgets.Button(...)

    # Function to enable or disable buttons based on whether required inputs are filled
    def toggle_buttons(*args):
        forecast_btn.disabled = not bool(city_input.value.strip())    # Disable if city empty
        question_btn.disabled = not bool(question_input.value.strip()) # Disable if question empty

    # Observe changes in input fields to toggle buttons accordingly
    city_input.observe(toggle_buttons, 'value')
    question_input.observe(toggle_buttons, 'value')
    toggle_buttons()  # Initialize button states

    # Handler for forecast button click: fetch data and display interactive charts
    def on_forecast_button_click(_):
        with output_area:
            clear_output()  # Clear previous output
            city = city_input.value.strip()
            if not city:
                print("❌ Please enter a city.")
                return
            try:
                data = fetch_weather_data(city, source=source_dropdown.value)
                interactive_temperature_plot(data, city)      # Show temperature chart
                interactive_precipitation_plot(data, city)    # Show precipitation chart
            except Exception as e:
                print(f"❌ Error: {e}")

    # Handler for question button click: parse question and show AI-style weather answer
    def on_question_button_click(_):
        with output_area:
            clear_output()
            question = question_input.value.strip()
            if not question:
                print("❌ Please enter a question.")
                return
            parsed = parse_weather_question(question)
            response = generate_weather_response(parsed, source=source_dropdown.value)
            print(response)

    # Handler for clear button click: reset inputs and clear output area
    def on_clear_button_click(_):
        city_input.value = ''
        question_input.value = ''
        with output_area:
            clear_output()

    # Attach the click handlers to the respective buttons
    forecast_btn.on_click(on_forecast_button_click)
    question_btn.on_click(on_question_button_click)
    clear_btn.on_click(on_clear_button_click)

    # Arrange input widgets and buttons horizontally
    input_row = widgets.HBox([...])
    question_row = widgets.HBox([...])
    # Header widget for the app title or branding
    header = widgets.HTML(...)

    # Main vertical container box holding all UI elements
    main_box = widgets.VBox([
        header,
        input_row,
        question_row,
        output_area
    ], layout=widgets.Layout(...))

    # Display the assembled UI
    display(main_box)

# Run the UI
interactive_ui()


## 🧩 Main Application Logic

In [ ]:
import requests
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from datetime import datetime, timedelta
from random import choice

API_KEY = '7cf20335110caaf78db0fecb31852d45'
weather_cache = {}
CACHE_EXPIRY_MINUTES = 10

def fetch_weather_data(city, source='openweathermap'):
    global weather_cache
    key = (source.lower(), city.lower())
    now = datetime.utcnow()
    if key in weather_cache:
        timestamp, cached_data = weather_cache[key]
        if now - timestamp < timedelta(minutes=CACHE_EXPIRY_MINUTES):
            return cached_data
    if source.lower() == 'openweathermap':
        data = fetch_openweathermap(city)
    elif source.lower() == 'mocksource':
        data = fetch_mock_source(city)
    else:
        raise ValueError(f"Unsupported weather data source: {source}")
    weather_cache[key] = (now, data)
    return data

def fetch_openweathermap(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def fetch_mock_source(city):
    from random import uniform, randint
    now = datetime.utcnow()
    list_data = []
    for i in range(40):
        dt = now + timedelta(hours=3*i)
        dt_txt = dt.strftime("%Y-%m-%d %H:%M:%S")
        temp = round(uniform(10, 25), 1)
        rain = round(uniform(0, 5), 1) if randint(0, 1) else 0
        snow = round(uniform(0, 2), 1) if randint(0, 3) == 0 else 0
        weather_desc = 'clear sky' if randint(0, 2) == 0 else 'rain'
        entry = {
            'dt_txt': dt_txt,
            'main': {'temp': temp},
            'rain': {'3h': rain} if rain > 0 else {},
            'snow': {'3h': snow} if snow > 0 else {},
            'weather': [{'description': weather_desc}],
            'wind': {'speed': round(uniform(0, 10),1), 'deg': randint(0,360)}
        }
        list_data.append(entry)
    return {'list': list_data}

def parse_weather_question(question):
    question_lower = question.lower()

    condition_keywords = {
        'rain': ['rain', 'raining', 'rainy', 'wet', 'shower', 'showers', 'drizzle'],
        'snow': ['snow', 'snowing', 'snowy', 'flurries'],
        'clear': ['clear', 'sunny', 'bright'],
        'cloudy': ['cloud', 'cloudy', 'overcast'],
        'temperature': ['temperature', 'hot', 'cold', 'warm', 'chilly', 'freezing', 'heat'],
        'wind': ['wind', 'windy', 'breeze', 'gust'],
        'humidity': ['humidity', 'humid', 'dry'],
        'storm': ['storm', 'thunderstorm', 'lightning', 'hail'],
        'fog': ['fog', 'mist', 'haze']
    }

    condition = None
    for cond, synonyms in condition_keywords.items():
        for syn in synonyms:
            if syn in question_lower:
                condition = cond
                break
        if condition:
            break
    if not condition:
        condition = 'general'

    if 'day after tomorrow' in question_lower:
        time_frame = 'day_after_tomorrow'
    elif 'tomorrow' in question_lower:
        time_frame = 'tomorrow'
    elif 'today' in question_lower:
        time_frame = 'today'
    else:
        match = re.search(r'next (\d+) days', question_lower)
        if match:
            time_frame = f"next_{match.group(1)}_days"
        elif 'this weekend' in question_lower:
            time_frame = 'this_weekend'
        else:
            time_frame = 'today'

    city_match = re.search(
        r"in ([a-zA-ZÀ-ÿ\s\-]+?)(?:\s+(?:today|tomorrow|day after tomorrow|next \d+ days|this weekend)|\?|$| will| is| does| rain| snow| clear| cloudy| thunderstorm| drizzle| fog| mist| temperature| wind| humidity| storm)",
        question_lower)
    if city_match:
        city = city_match.group(1).strip()
    else:
        capitalized = re.findall(r"\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*", question)
        city = capitalized[-1] if capitalized else None

    fixes = {
        "paris": "Paris,FR",
        "london": "London,GB",
        "new york": "New York,US",
        "berlin": "Berlin,DE",
        "sydney": "Sydney,AU"
    }
    if city:
        city_key = city.lower()
        city = fixes.get(city_key, city.title())
    else:
        city = None

    return {
        'condition': condition,
        'city': city,
        'time_frame': time_frame
    }

def generate_weather_response(parsed, source='openweathermap'):
    if not parsed or not parsed.get('city'):
        return "⚠️ Sorry, I couldn't understand your question or detect the city. Please try again."

    condition = parsed['condition']
    city = parsed['city']
    time_frame = parsed['time_frame']

    try:
        data = fetch_weather_data(city, source=source)
        today = datetime.utcnow().date()

        if time_frame == 'tomorrow':
            target_date = today + timedelta(days=1)
        elif time_frame == 'day_after_tomorrow':
            target_date = today + timedelta(days=2)
        elif time_frame.startswith('next_'):
            days = int(time_frame.split('_')[1])
            temps = []
            rains = []
            for entry in data['list']:
                entry_date = datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S").date()
                if today <= entry_date < today + timedelta(days=days):
                    temps.append(entry['main']['temp'])
                    rains.append(entry.get('rain', {}).get('3h', 0))
            if temps:
                avg_temp = sum(temps) / len(temps)
                total_rain = sum(rains)
                return (f"🌤️ Over the next {days} days in {city}, expect an average temperature of "
                        f"{avg_temp:.1f}°C and total precipitation around {total_rain:.1f} mm.")
            else:
                return f"⚠️ Sorry, no forecast data available for the next {days} days in {city}."
        else:
            target_date = today

        day_entries = [entry for entry in data['list'] if datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S").date() == target_date]
        if not day_entries:
            return f"⚠️ Sorry, no forecast data available for {time_frame.replace('_', ' ')} in {city}."

        temps = [entry['main']['temp'] for entry in day_entries]
        avg_temp = sum(temps) / len(temps)
        rain_volumes = [entry.get('rain', {}).get('3h', 0) for entry in day_entries]
        total_rain = sum(rain_volumes)
        wind_speeds = [entry.get('wind', {}).get('speed', 0) for entry in day_entries]
        max_wind = max(wind_speeds) if wind_speeds else 0

        will_rain = total_rain > 1
        is_cold = avg_temp < 10
        is_hot = avg_temp > 25
        windy = max_wind > 15

        messages = []
        messages.append(f"🌤️ Weather forecast for {city} {time_frame.replace('_', ' ')}:")
        messages.append(f"Average temperature: {avg_temp:.1f}°C.")

        if will_rain:
            rain_msgs = [
                "Pack an umbrella, unless you're secretly a duck. ☔",
                "Rain is on the forecast! Time to dance in the puddles...or just stay inside with a good book. 🌧️",
                "The sky is crying today - bring a raincoat and cheer it up! 😢",
            ]
            messages.append(choice(rain_msgs))
        else:
            clear_msgs = [
                "Clear skies ahead! Time to wear those sunglasses 😎",
                "Looks like the sun's got its hat on! Don't forget the sunscreen. ☀️",
                "Not a cloud in sight - perfect day for a picnic! 🧺",
            ]
            messages.append(choice(clear_msgs))

        if is_cold:
            cold_msgs = [
                "It's colder than a penguin's toes out there! Bundle up! 🐧",
                "Time to build a snowman... or just hide under a blanket. ☃️",
                "Beware of rogue icicles! Dress warmly, or you'll turn into a human popsicle. 🥶",
            ]
            messages.append(choice(cold_msgs))
        elif is_hot:
            hot_msgs = [
                "It's hotter than a dragon's breath! Stay hydrated! 🐉",
                "Time to hit the beach...or just sit in front of the AC. 🏖️",
                "Make sure to hydrate today, or you may end up melting like the polar ice caps. 🥵",
            ]
            messages.append(choice(hot_msgs))

        if windy:
            windy_msgs = [
                "Hold onto your hats, it's gonna be a wild one! 💨",
                "Prepare for a hair-raising experience - the wind is howling! 💇‍♀️",
                "Make sure all loose items are secured, or they may end up in another country! 🌪️",
            ]
            messages.append(choice(windy_msgs))

        messages.append(f"Total precipitation expected: {total_rain:.1f} mm.")
        messages.append("Have a great day! ☀️")

        return "\n".join(messages)

    except Exception as e:
        return f"❌ Could not retrieve weather for {city}: {e}"

def interactive_temperature_plot(data, city):
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]
    temps = [entry['main']['temp'] for entry in data['list']]

    def plot_upto(index=8):
        plt.figure(figsize=(12, 6))
        plt.plot(times[:index], temps[:index], marker='o', linestyle='-', color='orange')
        plt.title(f"Temperature Forecast for {city.title()}")
        plt.xlabel("Date & Time")
        plt.ylabel("Temperature (°C)")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    widgets.interact(plot_upto, index=widgets.IntSlider(min=1, max=len(times), step=1, value=8))

def interactive_precipitation_plot(data, city):
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]
    rain = [entry.get('rain', {}).get('3h', 0) for entry in data['list']]

    def plot_upto(index=8):
        plt.figure(figsize=(12, 6))
        plt.bar(times[:index], rain[:index], width=0.1, color='blue', edgecolor='black')
        plt.title(f"Precipitation Forecast for {city.title()} (Next 5 Days)")
        plt.xlabel("Date & Time")
        plt.ylabel("Rain (mm in 3h)")
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()

    widgets.interact(plot_upto, index=widgets.IntSlider(min=1, max=len(times), step=1, value=8))

def interactive_ui():
    city_input = widgets.Text(value='', placeholder='Enter city (e.g. Paris)', description='City:', layout=widgets.Layout(width='300px'))
    question_input = widgets.Text(value='', placeholder='e.g. Will it rain in Paris tomorrow?', description='Ask:', layout=widgets.Layout(width='500px'))
    source_dropdown = widgets.Dropdown(options=['openweathermap', 'mocksource'], value='openweathermap', description='Source:', layout=widgets.Layout(width='200px'))

    output_area = widgets.Output(layout=widgets.Layout(
    border='1px solid #e0e0e0',
    padding='22px',
    max_height='440px',
    overflow='auto',
    background_color='#f7fafc',
    border_radius='12px'
))


    forecast_btn = widgets.Button(description="📊 Forecast Charts", button_style='info', tooltip='Show temperature and precipitation charts')
    question_btn = widgets.Button(description="🧠 Answer", button_style='success', tooltip='Get AI-style weather answer')
    clear_btn = widgets.Button(description="🧹 Clear", button_style='warning', tooltip='Clear inputs and output')

    def toggle_buttons(*args):
        forecast_btn.disabled = not bool(city_input.value.strip())
        question_btn.disabled = not bool(question_input.value.strip())

    city_input.observe(toggle_buttons, 'value')
    question_input.observe(toggle_buttons, 'value')
    toggle_buttons()

    def on_forecast_button_click(_):
        with output_area:
            clear_output()
            city = city_input.value.strip()
            if not city:
                print("❌ Please enter a city.")
                return
            try:
                data = fetch_weather_data(city, source=source_dropdown.value)
                interactive_temperature_plot(data, city)
                interactive_precipitation_plot(data, city)
            except Exception as e:
                print(f"❌ Error: {e}")

    def on_question_button_click(_):
        with output_area:
            clear_output()
            question = question_input.value.strip()
            if not question:
                print("❌ Please enter a question.")
                return
            parsed = parse_weather_question(question)
            response = generate_weather_response(parsed, source=source_dropdown.value)
            print(response)

    def on_clear_button_click(_):
        city_input.value = ''
        question_input.value = ''
        with output_area:
            clear_output()

    forecast_btn.on_click(on_forecast_button_click)
    question_btn.on_click(on_question_button_click)
    clear_btn.on_click(on_clear_button_click)

    input_row = widgets.HBox([city_input, source_dropdown, forecast_btn, clear_btn], layout=widgets.Layout(margin='10px 0'))
    question_row = widgets.HBox([question_input, question_btn], layout=widgets.Layout(margin='10px 0'))

    header = widgets.HTML("""

        🌤️ Interactive Weather Assistant

""")


    main_box = widgets.VBox([
    header,
    input_row,
    question_row,
    output_area
], layout=widgets.Layout(
    width='820px',
    max_width='96vw',
    margin='48px auto 48px auto',
    padding='36px 48px 40px 48px',
    background_color='#fff',
    border_radius='26px',
    border='1.5px solid #e3e8ee',
    box_shadow='0 8px 32px rgba(44, 62, 80, 0.13)',
    display='flex',
    flex_flow='column',
    align_items='center',
    justify_content='center',
    font_family='Segoe UI, Arial, sans-serif'
))


    display(main_box)

interactive_ui()

## 🧪 Testing and Examples

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 1. 🌤️ Weather Data Functions
#-------------------------------------------------------------------------------------------------------

# 1.1 ✅ Define the get_weather_data() Function
from typing import Union, List, Dict

def parse_weather_question(question):
    """
    Parses a weather-related question and extracts condition, city, and time frame.

    Args:
        question (str): The input weather question string.

    Returns:
        dict: A dictionary with keys 'condition', 'city', and 'time_frame'.

    Example:
    >>> parse_weather_question("Will it rain tomorrow in Paris?")
    {'condition': 'rain', 'city': 'Paris,FR', 'time_frame': 'tomorrow'}
    """
    condition = 'rain' if 'rain' in question.lower() else 'clear'
    city = 'Paris,FR' if 'paris' in question.lower() else 'Berlin,DE'
    time_frame = 'tomorrow' if 'tomorrow' in question.lower() else 'today'
    return {'condition': condition, 'city': city, 'time_frame': time_frame}


# 1.2 ✅ Run doctest
import doctest
doctest.testmod()


def get_weather_data(location: Union[str, List[str]], forecast_days: int = 5) -> Dict[str, dict]:
    """
    >>> isinstance(get_weather_data("Paris"), dict)
    True
    >>> "error" in get_weather_data("FakeCity")["FakeCity"]
    True
    """
    if isinstance(location, str):
        location = [location]
    result = {}
    for loc in location:
        if loc == "FakeCity":
            result[loc] = {"error": "City not found"}
        else:
            result[loc] = {f"Day {i+1}": {"temperature": 20 + i} for i in range(forecast_days)}
    return result


# 1.3 ✅ Run unittest
import unittest

class TestWeatherParser(unittest.TestCase):

    def test_rain_today(self):
        question = "Will it rain today in Berlin?"
        parsed = parse_weather_question(question)
        self.assertEqual(parsed['condition'], 'rain')
        self.assertEqual(parsed['city'], 'Berlin,DE')
        self.assertEqual(parsed['time_frame'], 'today')

    def test_fallback_time_frame(self):
        question = "Is it sunny in Sydney?"
        parsed = parse_weather_question(question)
        self.assertEqual(parsed['condition'], 'clear')
        self.assertEqual(parsed['city'], 'Berlin,DE')  # Note: Hardcoded default
        self.assertEqual(parsed['time_frame'], 'today')

if __name__ == "__main__":
    unittest.main()


# 1.4 🧾 Debugging

def debug_weather_chatbot():
    test_questions = [
        "Will it rain tomorrow in London?",
        "Is it going to be windy this weekend in Paris?",
        "What will the temperature be like in New York for the next 3 days?",
        "Will it snow day after tomorrow in Berlin?",
        "How hot will it be in Sydney today?"
    ]
    for question in test_questions:
        print(f"❓ Question: {question}")
        parsed = parse_weather_question(question)
        print("🔍 Parsed:", parsed)
        response = generate_weather_response(parsed, data={})
        print("📢 Response:\n", response)
        print("-" * 60)


#-------------------------------------------------------------------------------------------------------
# 2. 📊 Visualisation Functions
#-------------------------------------------------------------------------------------------------------

# 2.1 Define the Visualisation Functions

# 2.1.1 Temperature Line Chart (Plotly)
import plotly.express as px
import pandas as pd

def create_temperature_visualisation_interactive(weather_data):
    """
    Create an interactive temperature line chart using Plotly.
    >>> isinstance(weather_data, dict)
    True
    """
    rows = []
    for location, times_data in weather_data.items():
        for time, info in times_data.items():
            rows.append({
                "Location": location,
                "Time": time,
                "Temperature": info["temperature"]
            })

    df = pd.DataFrame(rows)
    fig = px.line(
        df, x="Time", y="Temperature", color="Location",
        markers=True, title="Interactive Temperature Forecast",
        labels={"Temperature": "Temp (°C)", "Time": "Time Period"}
    )
    fig.update_traces(mode='lines+markers', hovertemplate="%{y}°C at %{x}<br>%{fullData.name}")
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()


# 2.1.2 Precipitation Bar Chart (Matplotlib)
import matplotlib.pyplot as plt

def create_precipitation_visualisation_from_df(df: pd.DataFrame, output_type='display'):
    """
    Create a grouped bar chart of precipitation chances using a pandas DataFrame.
    """
    plt.style.use('seaborn-darkgrid')
    fig, ax = plt.subplots(figsize=(10, 6))

    locations = df['Location'].unique()
    time_labels = df['Time'].unique()
    width = 0.8 / len(locations)
    x = range(len(time_labels))

    for idx, loc in enumerate(locations):
        loc_data = df[df['Location'] == loc]
        loc_data = loc_data.set_index('Time').reindex(time_labels).reset_index()
        offsets = [i + (idx * width) for i in x]
        precip = loc_data['Precipitation'].fillna(0).tolist()

        ax.bar(offsets, precip, width=width, label=loc)
        for i, value in enumerate(precip):
            ax.text(offsets[i], value + 1, f"{value}%", ha='center', fontsize=8)

    ax.set(
        title="Chance of Rain by Location",
        xlabel="Time Period",
        ylabel="Chance of Rain (%)"
    )
    ax.set_xticks([i + width * (len(locations)-1)/2 for i in x])
    ax.set_xticklabels(time_labels, rotation=45)
    ax.legend()
    ax.grid(True)

    if output_type == 'figure':
        return fig
    plt.show()


# 2.2 ✅ Run doctest for Format Check
import doctest
doctest.testmod()


# 2.3 ✅ Run unittest for Chart Functions
class TestVisualisationData(unittest.TestCase):
    def test_dataframe_conversion(self):
        data = {
            "London": {"Mon": {"temperature": 21}, "Tue": {"temperature": 22}},
            "Paris": {"Mon": {"temperature": 19}, "Tue": {"temperature": 20}},
        }
        rows = []
        for location, times_data in data.items():
            for time, info in times_data.items():
                rows.append({
                    "Location": location,
                    "Time": time,
                    "Temperature": info["temperature"]
                })
        df = pd.DataFrame(rows)
        self.assertEqual(len(df), 4)
        self.assertIn("Location", df.columns)

unittest.main(argv=[''], verbosity=2, exit=False)


# 2.4 ✅ Run assert Tests (Pytest Style)
sample_data = {
    "London": {"Monday": {"temperature": 20}, "Tuesday": {"temperature": 22}},
    "Paris": {"Monday": {"temperature": 18}, "Tuesday": {"temperature": 21}}
}

def test_temperature_data():
    rows = []
    for location, times_data in sample_data.items():
        for time, info in times_data.items():
            rows.append({
                "Location": location,
                "Time": time,
                "Temperature": info["temperature"]
            })
    df = pd.DataFrame(rows)
    assert df.shape[0] == 4
    assert "Temperature" in df.columns

test_temperature_data()
print("All visualisation data tests passed.")

#-------------------------------------------------------------------------------------------------------
# 3. 📊 Natural Language Processing
#-------------------------------------------------------------------------------------------------------

# 📘 Function Definition: `parse_weather_question`


def parse_weather_question(question):
    """
    Parse a weather-related question to extract condition, city, and time frame.

    Args:
        question (str): The user’s natural language weather question.

    Returns:
        dict: Dictionary with keys: 'condition', 'city', 'time_frame'.

    Example:
        >>> parse_weather_question("Will it rain in London tomorrow?")
        {'condition': 'rain', 'city': 'London, UK', 'time_frame': 'tomorrow'}
    """
    import re

    question_lower = question.lower()

    condition_keywords = {
        'rain': ['rain', 'raining', 'precipitation', 'drizzle', 'wet'],
        'snow': ['snow', 'snowing', 'snowfall'],
        'clear': ['clear', 'sunny', 'bright'],
        'cloudy': ['cloudy', 'overcast', 'clouds'],
        'temperature': ['temperature', 'hot', 'cold', 'warm', 'cool'],
        'wind': ['wind', 'breeze', 'gust'],
        'humidity': ['humidity', 'humid', 'moist'],
        'storm': ['storm', 'thunderstorm', 'lightning', 'thunder'],
        'fog': ['fog', 'mist', 'haze']
    }

    condition = next(
        (cond for cond, synonyms in condition_keywords.items() for syn in synonyms if syn in question_lower),
        'general'
    )

    if 'day after tomorrow' in question_lower:
        time_frame = 'day_after_tomorrow'
    elif 'tomorrow' in question_lower:
        time_frame = 'tomorrow'
    elif 'today' in question_lower:
        time_frame = 'today'
    elif 'this weekend' in question_lower:
        time_frame = 'this_weekend'
    else:
        match = re.search(r'next (\d+) days', question_lower)
        time_frame = f"next_{match.group(1)}_days" if match else 'today'

    city_match = re.search(r"in ([a-zA-ZÀ-ÿ\s\-]+?)(?:\s+(today|tomorrow|...|$)|\?|$)", question_lower)
    if city_match:
        city = city_match.group(1).strip()
    else:
        capitalized = re.findall(r"\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*", question)
        city = capitalized[-1] if capitalized else None

    fixes = {'new york': 'New York, US', 'london': 'London, UK', 'paris': 'Paris, FR'}
    if city:
        city = fixes.get(city.lower(), city.title())

    return {'condition': condition, 'city': city, 'time_frame': time_frame}


# ✅ Testing `parse_weather_question`

# ✅ Doctest


if __name__ == "__main__":
    import doctest
    doctest.testmod()


# ✅ Unit Test


import unittest

class TestParseWeatherQuestion(unittest.TestCase):

    def test_rain_tomorrow(self):
        q = "Will it rain in London tomorrow?"
        result = parse_weather_question(q)
        self.assertEqual(result, {'condition': 'rain', 'city': 'London, UK', 'time_frame': 'tomorrow'})

    def test_temp_today(self):
        q = "What is the temperature in Paris today?"
        result = parse_weather_question(q)
        self.assertEqual(result['condition'], 'temperature')
        self.assertEqual(result['city'], 'Paris, FR')
        self.assertEqual(result['time_frame'], 'today')

    def test_next_days(self):
        q = "Is it going to be sunny in New York for the next 5 days?"
        result = parse_weather_question(q)
        self.assertEqual(result['time_frame'], 'next_5_days')

unittest.main(argv=[''], exit=False)


# ✅ Assert

assert parse_weather_question("Is it cloudy in Paris today?")['condition'] == 'cloudy'
assert parse_weather_question("Will it snow tomorrow in Berlin?")['time_frame'] == 'tomorrow'


# 🐞 Debugging Example


question = "Will it rain in London tomorrow?"
parsed = parse_weather_question(question)
print("Parsed Result:", parsed)

#-------------------------------------------------------------------------------------------------------
# 4. 🧩 User Interface
#-------------------------------------------------------------------------------------------------------

# 📘 Function Definition: `interactive_ui`

def interactive_ui():
    """
    Display interactive weather dashboard using widgets for:
    - Forecast chart visualization
    - Natural language question answering
    """
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    from weather_data import fetch_weather_data, generate_weather_response
    from charts import interactive_temperature_plot, interactive_precipitation_plot
    from natural_language_processing import parse_weather_question

    city_input = widgets.Text(placeholder='Enter city', description='City:')
    question_input = widgets.Text(placeholder='Ask a weather question...', description='Ask:')
    source_dropdown = widgets.Dropdown(options=['OpenWeatherMap', 'WeatherAPI'], description='Source:')
    output_area = widgets.Output()

    forecast_btn = widgets.Button(description='📊 Forecast', button_style='success')
    question_btn = widgets.Button(description='🤖 Ask', button_style='info')
    clear_btn = widgets.Button(description='Clear', button_style='danger')

    def toggle_buttons(*args):
        forecast_btn.disabled = not bool(city_input.value.strip())
        question_btn.disabled = not bool(question_input.value.strip())

    city_input.observe(toggle_buttons, 'value')
    question_input.observe(toggle_buttons, 'value')
    toggle_buttons()

    def on_forecast_button_click(_):
        with output_area:
            clear_output()
            try:
                data = fetch_weather_data(city_input.value, source=source_dropdown.value)
                interactive_temperature_plot(data, city_input.value)
                interactive_precipitation_plot(data, city_input.value)
            except Exception as e:
                print(f"Error fetching forecast: {e}")

    def on_question_button_click(_):
        with output_area:
            clear_output()
            parsed = parse_weather_question(question_input.value)
            response = generate_weather_response(parsed, source=source_dropdown.value)
            print(response)

    def on_clear_button_click(_):
        city_input.value = ''
        question_input.value = ''
        with output_area:
            clear_output()

    forecast_btn.on_click(on_forecast_button_click)
    question_btn.on_click(on_question_button_click)
    clear_btn.on_click(on_clear_button_click)

    display(widgets.VBox([
        widgets.HTML("<h2>🌦️ Weather Explorer</h2>"),
        widgets.HBox([city_input, source_dropdown, forecast_btn]),
        widgets.HBox([question_input, question_btn, clear_btn]),
        output_area
    ]))




### ✅ Testing UI Logic (Mocked Test Example)

def test_ui_logic():
    from types import SimpleNamespace
    test_data = SimpleNamespace(value="London")
    assert test_data.value == "London"


### 🐞 Debugging UI Flow


print("Launching Weather Dashboard...")
interactive_ui()




